In [63]:
import geopandas as gpd
import numpy as np

In [71]:
def get_total_agrupado(data, groupby_factors, cols_to_drop=[]):

    total = data.drop(
        columns=cols_to_drop).groupby(by=groupby_factors).sum().reset_index()
    
    return total

In [72]:
def get_area_corrigida(data_desm, data_res):
    final = data_desm.merge(
        data_res.rename(columns={'area_km': 'area_res_km'}),
        left_on=['year', 'state'],
        right_on=['year', 'state'],
        how='outer',
    )
    
    final.area_res_km = final.area_res_km.replace(np.nan, 0)
    final['true_area_km'] = final['area_km'] - final['area_res_km']


    return final


In [81]:
base_input_path = 'D:/OneDrive/dados/raw_data/inpe/prodes/'
file_prodes_amazonia = 'prodes_amazonia_nb/prodes_amazonia_nb.gpkg'
file_prodes_cerrado = 'prodes_cerrado_nb/prodes_cerrado_nb.gpkg'


base_output_path = ('D:/OneDrive/documentos/2024-01/tutoria_unifesp/projeto'
                    + '/projeto_grupo4/dados/')


read_cols = ['state', 'sub_class', 'year', 'area_km']

### Amazônia

In [23]:
amazonia_desm = gpd.read_file(base_input_path + file_prodes_amazonia,
                              layer='yearly_deforestation_biome',
                              columns=read_cols, ignore_geometry=True)

amazonia_res = gpd.read_file(base_input_path + file_prodes_amazonia,
                             layer='residual_biome',
                             columns=read_cols, ignore_geometry=True)

In [28]:
amazonia_desm.loc[amazonia_desm.sub_class.str.startswith('d'),
                  'sub_class'] = 'nao_classificado'

In [75]:
desm_total_amaz = get_total_agrupado(amazonia_desm,
                                     groupby_factors=['year', 'state'],
                                     cols_to_drop=['sub_class'])


res_total_amaz = get_total_agrupado(amazonia_res,
                                     groupby_factors=['year', 'state'])

amaz_nao_corrigido_subclassificado = get_total_agrupado(
    amazonia_desm,
    groupby_factors=['year', 'state', 'sub_class'],
)

In [73]:
amaz_corrigido = get_area_corrigida(desm_total_amaz, res_total_amaz)

In [74]:
amaz_corrigido

,year,state,area_km,area_res_km,true_area_km
0,2008.0,AC,288.792419,0.000000,288.792419
1,2008.0,AM,612.131272,0.000000,612.131272
2,2008.0,AP,99.915680,0.000000,99.915680
3,2008.0,MA,725.113003,0.000000,725.113003
4,2008.0,MT,3303.822938,0.000000,3303.822938
...,...,...,...,...,...
139,2023.0,MT,1905.109009,8.455357,1896.653652
140,2023.0,PA,2883.614095,32.338522,2851.275572
141,2023.0,RO,794.333045,7.466947,786.866098
142,2023.0,RR,276.009334,9.723526,266.285808


In [76]:
amaz_nao_corrigido_subclassificado

,year,state,sub_class,area_km
0,2008.0,AC,nao_classificado,288.792419
1,2008.0,AM,nao_classificado,612.131272
2,2008.0,AP,nao_classificado,99.915680
3,2008.0,MA,nao_classificado,725.113003
4,2008.0,MT,nao_classificado,3303.822938
...,...,...,...,...
196,2023.0,RR,mineração,0.604164
197,2023.0,RR,nao_classificado,17.333651
198,2023.0,TO,corte raso com solo exposto,4.383690
199,2023.0,TO,corte raso com vegetação,1.070979


In [85]:
amaz_corrigido.to_csv(
    base_output_path + 'amazonia_corrigido_dados_agrupados.csv',
    index=False
)

amaz_nao_corrigido_subclassificado.to_csv(
    base_output_path + 'amazonia_nao_corrigido_subclassificado_dados_agrupados.csv',
    index=False,
)

### Cerrado

In [83]:
cerrado_desm = gpd.read_file(base_input_path + file_prodes_cerrado,
                              layer='yearly_deforestation',
                              columns=read_cols, ignore_geometry=True)

cerrado_res = gpd.read_file(base_input_path + file_prodes_cerrado,
                             layer='residual',
                             columns=read_cols, ignore_geometry=True)

In [87]:
desm_total_cerr = get_total_agrupado(cerrado_desm,
                                     groupby_factors=['year', 'state'])


res_total_cerr = get_total_agrupado(cerrado_res,
                                     groupby_factors=['year', 'state'])

In [88]:
cerr_corrigido = get_area_corrigida(desm_total_cerr, res_total_cerr)

In [89]:
cerr_corrigido

,year,state,area_km,area_res_km,true_area_km
0,2002.0,BAHIA,2415.154820,0.000000,2415.154820
1,2002.0,DISTRITO FEDERAL,25.562308,0.000000,25.562308
2,2002.0,GOIÁS,13339.586737,0.000000,13339.586737
3,2002.0,MARANHÃO,4669.892457,0.000000,4669.892457
4,2002.0,MATO GROSSO,9415.795091,0.000000,9415.795091
...,...,...,...,...,...
221,2023.0,PARÁ,245.795962,17.545543,228.250418
222,2023.0,PIAUÍ,1127.508592,30.096389,1097.412203
223,2023.0,RONDÔNIA,12.990353,0.082338,12.908015
224,2023.0,SÃO PAULO,1.481144,0.150319,1.330825


In [90]:
cerr_corrigido.to_csv(
    base_output_path + 'cerrado_corrigido_dados_agrupados.csv',
    index=False
)